In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import os
import logging
# Function to send a query and get the response
import requests
import json

# Define the URL of your FastAPI server
BASE_URL = "http://localhost:8000"  # Adjust this if your server is running on a different port or host

# Define the endpoint
ENDPOINT = f"{BASE_URL}/api/langchain/interactive_langgraph_query/query"

def send_query(query, model="ollama"):
    payload = {"query": query, "model": model}
    headers = {"Content-Type": "application/json"}
    print(f"Sending query to {ENDPOINT} with payload: {payload}")
    
    try:
        response = requests.post(ENDPOINT, json=payload, headers=headers)
        response.raise_for_status()
        print(f"Received response from {ENDPOINT}: {response.json()}")
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error sending query to {ENDPOINT}: {str(e)}")
        return {"error": str(e)}

def test_simple_queries(model="openai"):
    queries = [
        "What is the history of Maidstone, England?"
    ]
    
    print(f"Testing simple queries using {model} model:")
    for query in queries:
        print(f"\nQuery: {query}")
        result = send_query(query, model)
        print("Response:")
        print(json.dumps(result, indent=2))
        print("=" * 50)

def test_followup_queries(model="openai"):
    queries = [
        "What is the latest local news from a particular town?"
    ]
    
    print(f"Testing queries requiring follow-up using {model} model:")
    for query in queries:
        print(f"\nInitial Query: {query}")
        result = send_query(query, model)
        print("Initial Response:")
        print(json.dumps(result, indent=2))
        
        follow_up_count = 0
        max_follow_ups = 3
        
        while result.get("needs_more_info", False) and follow_up_count < max_follow_ups:
            follow_up = input("Please provide more information: ")
            follow_up_query = f"{query} {follow_up}"
            follow_up_result = send_query(follow_up_query, model)
            print(f"\nFollow-up Response {follow_up_count + 1}:")
            print(json.dumps(follow_up_result, indent=2))
            
            result = follow_up_result
            follow_up_count += 1
        
        if follow_up_count == max_follow_ups:
            print("\nMaximum number of follow-ups reached. Moving to next query.")
        elif not result.get("needs_more_info", False):
            print("\nFinal Response:")
            print(json.dumps(result, indent=2))
        
        print("=" * 50)

# Run the tests
#print("Running simple query tests with Ollama:\n")
#test_simple_queries("ollama")

print("\nRunning simple query tests with OpenAI:\n")
test_simple_queries("openai")

#print("\nRunning follow-up query tests with Ollama:\n")
#test_followup_queries("ollama")

#print("\nRunning follow-up query tests with OpenAI:\n")
#test_followup_queries("openai")


Running simple query tests with OpenAI:

Testing simple queries using openai model:

Query: What is the history of Maidstone, England?
Sending query to http://localhost:8000/api/langchain/interactive_langgraph_query/query with payload: {'query': 'What is the history of Maidstone, England?', 'model': 'openai'}


ERROR:root:Error sending query to http://localhost:8000/api/langchain/interactive_langgraph_query/query: 500 Server Error: Internal Server Error for url: http://localhost:8000/api/langchain/interactive_langgraph_query/query


Response:
{
  "error": "500 Server Error: Internal Server Error for url: http://localhost:8000/api/langchain/interactive_langgraph_query/query"
}
